In [2]:
import pandas as pd 
from sqlalchemy import create_engine

USER = "postgres"
SENHA = "postgres"
HOST = "localhost"
DATABASE = "ada"

engine = create_engine(f"postgresql+psycopg2://{USER}:{SENHA}@{HOST}/{DATABASE}")

---

```
CREATE TABLE cars.category (
	category_id CHAR(3),
	employee_id INTEGER,
	category_name VARCHAR(50),
	rating VARCHAR(5),
	quantity_sold INTEGER,
	being_manufactured BOOLEAN,
	total_sold_value INTEGER
);

CREATE TABLE cars.customer (
	customer_id INTEGER,
	employee_id INTEGER,
	firstname VARCHAR(30),
	lastname VARCHAR(30),
	dof DATE,
	phone CHAR(10),
	email VARCHAR(50),
	city VARCHAR(255),
	country VARCHAR(255)
);

CREATE TABLE cars.employee (
	employee_id INTEGER,
	store_id INTEGER,
	firstname VARCHAR(30),
	lastname VARCHAR(30),
	dof DATE,
	phone CHAR(10),
	email VARCHAR(50),
	status VARCHAR(6),
	salary INTEGER,
	street VARCHAR(255),
	city VARCHAR(255),
	country VARCHAR(255)
);

CREATE TABLE cars.order (
	order_id VARCHAR(6),
	customer_id INTEGER,
	product_id INTEGER,
	store_id INTEGER,
	employee_id INTEGER,
	order_date DATE,
	shipped_date DATE,
	required_date DATE,
	order_status VARCHAR(9),
	quantity INTEGER
);

CREATE TABLE cars.product (
	product_id INTEGER,
	provider_id CHAR(6),
	category_id CHAR(3),
	product_name VARCHAR(30),
	model VARCHAR(30),
	year CHAR(4),
	color VARCHAR(30),
	km INTEGER,
	price INTEGER
);

CREATE TABLE cars.provider (
	provider_id CHAR(6),
	employee_id INTEGER,
	provider_name VARCHAR(50),
	debt INTEGER,
	phone CHAR(10),
	email VARCHAR(255),
	city VARCHAR(255),
	country VARCHAR(255)
);

CREATE TABLE cars.rise_archive (
	rise_id CHAR(4),
	amount_by_percent INTEGER,
	rise_date DATE,
	rise_state CHAR(9)
);

CREATE TABLE cars.store (
	store_id INTEGER,
	employee_id INTEGER,
	store_name VARCHAR(255),
	phone CHAR(10),
	street VARCHAR(255),
	city VARCHAR(255),
	country VARCHAR(255),
	email VARCHAR(255),
	post_code CHAR(4)
)
```

### Ferramentas:
- Git 
- PandaSQL 
- PostgreSQL 
- pgAdmin4

### Referências:
- https://pandas.pydata.org
- https://www.postgresql.org
- https://pypi.org/project/pandasql/
- https://pypi.org/project/psycopg2/
- https://www.kaggle.com/datasets/grncode/automotive-sector-compherensive-business-data-2023
- https://learnsql.com/blog/how-to-import-csv-to-postgresql/

---

<div style="background-color: Purple; padding: 10px;">
    <h2> Exploração Inicial dos Dados
</div>

- Tabela ``cars.product``;

In [5]:
query = """
SELECT * FROM cars.product
"""

df = pd.read_sql_query(query, con = engine)
df.head()

,product_id,provider_id,category_id,product_name,model,year,color,km,price
0,2000,Pr7670,MTG,Toyota,Camry,2019,DarkBlue,254760,21475
1,2001,Pr7893,VES,Ford,Mustang,2014,Turquoise,107930,40858
2,2002,Pr8017,QRQ,Honda,Civic,2000,Snow,265553,42863
3,2003,Pr8030,UFM,Chevrolet,Corvette,2020,OrangeRed,28751,16273
4,2004,Pr7793,DJO,BMW,3 Series,2015,LightSkyBlue,5836,27941


In [6]:
query = """
SELECT * FROM cars.product
"""

df = pd.read_sql_query(query, con = engine)
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 16723 entries, 0 to 16722
Data columns (total 9 columns):
 #   Column        Non-Null Count  Dtype 
---  ------        --------------  ----- 
 0   product_id    16723 non-null  int64 
 1   provider_id   16723 non-null  object
 2   category_id   16723 non-null  object
 3   product_name  16723 non-null  object
 4   model         16723 non-null  object
 5   year          16723 non-null  object
 6   color         16723 non-null  object
 7   km            16723 non-null  int64 
 8   price         16723 non-null  int64 
dtypes: int64(3), object(6)
memory usage: 1.1+ MB


Cada carro tem um product_id, com nome, modelo, ano e cor. 

In [20]:
query = """
SELECT product_id, c.category_id, category_name, product_name, model, year, color
FROM cars.product p
JOIN cars.category c ON c.category_id = p.category_id
WHERE product_name = 'Honda' and model = 'Civic'
ORDER BY category_name
"""

df = pd.read_sql_query(query, con = engine)
df

,product_id,category_id,category_name,product_name,model,year,color
0,16366,ARB,Convertible,Honda,Civic,1998,MistyRose
1,6448,OZQ,Convertible,Honda,Civic,1996,Aquamarine
2,10552,RGY,Convertible,Honda,Civic,2004,CornflowerBlue
3,16708,KFP,Convertible,Honda,Civic,1998,Turquoise
4,14656,OXW,Convertible,Honda,Civic,2015,DarkSeaGreen
5,18076,GLR,Convertible,Honda,Civic,2015,CadetBlue
6,13972,LMJ,Convertible,Honda,Civic,2009,AliceBlue
7,11578,QXM,Coupe,Honda,Civic,2003,BurlyWood
8,12262,KOQ,Coupe,Honda,Civic,2004,DeepPink
9,8158,FKS,Coupe,Honda,Civic,2011,Brown


---

- Tabela ``cars.provider``;

- Tabela ``cars.rise_archive``;

- Tabela ``cars.store``;